In [1]:
import os
import json
import tensorflow as tf
from keras.utils.layer_utils import count_params
from model import get_model

from data import get_dataset, get_zoo_elephants_images_and_labels, get_ELEP_images_and_labels
from train import SiameseModel
from metrics import get_kernel_mask, val, far, pairwise_accuracy

from tensorflow.keras import optimizers

In [98]:
im_size=256
base_model = tf.keras.applications.ResNet50V2(include_top=False,
                                                  weights="imagenet",
                                                  input_shape=(im_size, im_size, 3))
    
    
    
    
base_model.trainable = False   
trainable_count = count_params(base_model.trainable_weights)
trainable_count

0

In [99]:
base_model.trainable = True
trainable = False
for layer in base_model.layers:
    if layer.name == "conv5_block1_out":
        trainable = True
    layer.trainable = trainable
trainable_count = count_params(base_model.trainable_weights)
trainable_count

8933376

In [91]:
[(layer.name, layer.trainable) for layer in base_model.layers][-20:]

[('conv5_block2_1_relu', False),
 ('conv5_block2_2_pad', False),
 ('conv5_block2_2_conv', False),
 ('conv5_block2_2_bn', False),
 ('conv5_block2_2_relu', False),
 ('conv5_block2_3_conv', False),
 ('conv5_block2_out', True),
 ('conv5_block3_preact_bn', True),
 ('conv5_block3_preact_relu', True),
 ('conv5_block3_1_conv', True),
 ('conv5_block3_1_bn', True),
 ('conv5_block3_1_relu', True),
 ('conv5_block3_2_pad', True),
 ('conv5_block3_2_conv', True),
 ('conv5_block3_2_bn', True),
 ('conv5_block3_2_relu', True),
 ('conv5_block3_3_conv', True),
 ('conv5_block3_out', True),
 ('post_bn', True),
 ('post_relu', True)]

In [ ]:
base_model.summary()

In [101]:
base_model.trainable = False
inputs = tf.keras.Input(shape=(im_size,im_size, 3))
x = base_model(inputs, training=False)
conv1x1 = tf.keras.layers.Conv2D(kernel_size=1, filters=256, activation='relu')(x)
dw_conv_final = tf.keras.layers.DepthwiseConv2D(kernel_size=(8,8), activation='relu')(conv1x1)
# dw_conv2 = tf.keras.layers.DepthwiseConv2D(kernel_size=(3,3), activation='relu')(dw_conv1)
# dw_conv_final = tf.keras.layers.DepthwiseConv2D(kernel_size=(3,3), activation='relu')(dw_conv2)
# dw_conv4 = tf.keras.layers.DepthwiseConv2D(kernel_size=(3,3), activation='relu')(dw_conv3)
flatten = tf.keras.layers.Flatten()(dw_conv_final)
embedding_layer = tf.keras.layers.Dense(units=256)(flatten)
model = tf.keras.Model(inputs, embedding_layer)
model.summary()

Model: "model_49"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_71 (InputLayer)       [(None, 256, 256, 3)]     0         
                                                                 
 resnet50v2 (Functional)     (None, 8, 8, 2048)        23564800  
                                                                 
 conv2d_37 (Conv2D)          (None, 8, 8, 256)         524544    
                                                                 
 depthwise_conv2d_67 (Depthw  (None, 1, 1, 256)        16640     
 iseConv2D)                                                      
                                                                 
 flatten_17 (Flatten)        (None, 256)               0         
                                                                 
 dense_29 (Dense)            (None, 256)               65792     
                                                          

In [76]:
d=256
use_avg_pool=True


base_model.trainable = False

inputs = tf.keras.Input(shape=(im_size,im_size, 3))
x = base_model(inputs, training=False)
if use_avg_pool:
    # flatten = tf.keras.layers.Flatten()(x)
    flatten = tf.keras.layers.GlobalAveragePooling2D()(x)
else:
    dw_conv = tf.keras.layers.DepthwiseConv2D(kernel_size=(8,8), activation='relu')(x)
    flatten = tf.keras.layers.Flatten()(dw_conv)
dropout1 = tf.keras.layers.Dropout(rate=0.3)(flatten)

if d < 256:
    dense1 = tf.keras.layers.Dense(units=512, activation='relu')(dropout1)
    dropout2 = tf.keras.layers.Dropout(rate=0.3)(dense1)
    embedding_layer = tf.keras.layers.Dense(units=d, activation='relu')(dropout2)
else:
    embedding_layer = tf.keras.layers.Dense(units=d, activation='relu')(dropout1)


model = tf.keras.Model(inputs, embedding_layer)
trainable_count = count_params(model.trainable_weights)
trainable_count

524544

In [77]:
model.summary()

Model: "model_47"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_63 (InputLayer)       [(None, 256, 256, 3)]     0         
                                                                 
 resnet50v2 (Functional)     (None, 8, 8, 2048)        23564800  
                                                                 
 global_average_pooling2d_14  (None, 2048)             0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dropout_20 (Dropout)        (None, 2048)              0         
                                                                 
 dense_27 (Dense)            (None, 256)               524544    
                                                                 
Total params: 24,089,344
Trainable params: 524,544
Non-trainable params: 23,564,800
________________________________________

In [12]:
85133952/1247872

68.22330495435429

In [18]:
42600704/657664

64.7757882444531

In [44]:
model.summary()

Model: "model_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_24 (InputLayer)       [(None, 256, 256, 3)]     0         
                                                                 
 resnet50v2 (Functional)     (None, 8, 8, 2048)        23564800  
                                                                 
 depthwise_conv2d_20 (Depthw  (None, 1, 1, 2048)       133120    
 iseConv2D)                                                      
                                                                 
 flatten_15 (Flatten)        (None, 2048)              0         
                                                                 
 dropout_18 (Dropout)        (None, 2048)              0         
                                                                 
 dense_29 (Dense)            (None, 512)               1049088   
                                                          

In [1]:
model

NameError: name 'model' is not defined

In [2]:
with open('hyperparameters/initial_run.json', 'rb') as f:
    params = json.load(f)

In [3]:
params

{'image_size': 256,
 'batch_size': {'train': 128, 'val': 512},
 'margin': 0.5,
 'squared': False,
 'embedding_size': 256,
 'use_avg_pool': False,
 'dropout1_rate': 0.3,
 'dropout2_rate': 0.3,
 'lr': 0.0001,
 'decay_steps': 13,
 'decay_rate': 0.96,
 'dense_l2_reg_c': 0.01,
 'metrics_d': 1.25,
 'triplet_strategy': 'batch_all'}

In [4]:
siamese_model = SiameseModel(params, False)
siamese_model.compile(optimizer=optimizers.Adam(params['lr']))

In [5]:
data_dir = '/Users/deepakduggirala/Documents/ELPephant-cropped'

In [6]:
cache_files = {
        'train': 'train.cache',
        'val': 'val.cache'
    }

In [7]:
train_ds, N_train = get_dataset(get_ELEP_images_and_labels, params, data_dir, 'train', cache_files)
val_ds, N_val = get_dataset(get_ELEP_images_and_labels, params, data_dir, 'val', cache_files)

reading Tensor("args_0:0", shape=(), dtype=string)
reading Tensor("args_0:0", shape=(), dtype=string)


In [8]:
params['batch_size']['train'] = 50
params['batch_size']['val'] = 50

In [ ]:
siamese_model.fit(train_ds.take(1),
                      epochs=4,
                      validation_data=val_ds.take(1))